# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-oss:20b'
openai = OpenAI(base_url=os.getenv('OLLAMA_BASE_URL'), api_key=os.getenv('OLLAMA_API_KEY'))

There might be a problem with your API key? Please visit the troubleshooting notebook!


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2025/11/11/

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'curriculum page', 'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'product page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'product page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'social profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'social profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'social profile',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [10]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-oss:20b
Found 10 relevant links


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'portfolio page', 'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'product page', 'url': 'https://nebula.io/'},
  {'type': 'product page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'product page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'linkedin', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [11]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-oss:20b
Found 8 relevant links


{'links': [{'type': 'about page', 'url': 'https://huggingface.co/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'brand assets page', 'url': 'https://huggingface.co/brand'},
  {'type': 'learn page', 'url': 'https://huggingface.co/learn'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [12]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [ ]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

In [20]:
#brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
 You are an assistant that analyzes the contents of several relevant pages from a company website
 and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
 Respond in markdown without code blocks.
 Include details of company culture, customers and careers/jobs if you have the information.
 """


In [14]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [15]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-oss:20b
Found 3 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nLightricks/LTX-2\nUpdated\n6 days ago\n•\n861k\n•\n902\nfal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA\nUpdated\n6 days ago\n•\n30.5k\n•\n552\ntencent/HY-MT1.5-1.8B\nUpdated\n13 days ago\n•\n11.7k\n•\n745\nnvidia/nemotron-speech-streaming-en-0.6b\nUpdated\n8 days ago\n•\n3.16k\n•\n339\nLiquidAI/LFM2.5-1.2B-Instruct\nUpdated\n5 days ago\n•\n15.1k\n•\n311\nBrowse 2M+ models\nSpaces\nRunning\no

In [16]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("HuggingFace", "https://huggingface.co")

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [21]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [18]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-oss:20b
Found 10 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Hugging Face – The AI Community Building the Future  

**Tagline:** *“The platform where the machine learning community collaborates on models, datasets, and applications.”*  

---

## Overview
- **Founded**: 2016 (inferred from the long‑standing presence and community growth).  
- **Mission**: Democratize AI by making state‑of‑the‑art machine‑learning assets—models, datasets, and applications—open, shareable, and easy to deploy.  
- **Scale**: Over **2 M+ models**, **1 M+ applications** (“Spaces”), and **500 k+ datasets** spanning text, image, video, audio, and even 3‑D modalities.

---

## Core Platform

| Component | What it offers | Highlights |
|-----------|----------------|------------|
| **Models** | Pre‑trained weights for NLP, vision, audio, multimodal tasks. | 100 k+ public models, active community updates. |
| **Datasets** | Curated, versioned collections for training and evaluation. | 500 k+ datasets, 100 k+ downloads per month. |
| **Spaces** | Front‑end demos/run‑time apps using Gradio / Streamlit. | Instant deployment, 1 M+ app usages, zero‑cost “Run on Zero.” |
| **Docs & Community** | APIs, tutorials, forums, Discord, Slack. | Global community of researchers, hobbyists, and enterprises. |
| **HF Open Source Stack** | End‑to‑end tools: `transformers`, `datasets`, `accelerate`, `tokenizers`. | Unified, modular, and free for all. |

---

## Enterprise & Team Solutions

- **Pricing**: *Team* subscription at **$20/user/month** (self‑hosted). Enterprise contracts available on request.  
- **Security**:  
  - **Single Sign‑On** integration with SAML/OIDC.  
  - **Region control**: choose audit‑ready data locations.  
  - **Audit logs** for compliance.  
- **Managed Compute**: On‑premise or cloud resources for production workloads.  
- **Support**: Dedicated enterprise support and access to premium model hosting.  

---

## Community Culture

- **Open‑Source First**: All core libraries and many models are released under permissive licenses.  
- **Collaboration‑Driven**: Pull requests, issue discussions, community challenges keep the ecosystem vibrant.  
- **Inclusive Growth**: Mentorship through hackathons, webinars, and local meet‑ups.  

---

## Customers & Partners

- **Developers & Researchers**: Build, experiment, and iterate with instant API access.  
- **Startups & SMEs**: Accelerate ML product building with free layers of compute.  
- **Large Enterprises**: Scale AI solutions with secure, managed infrastructure and enterprise contracts.  
- **Academic & Research Institutions**: Share reproducible research through publicly hosted models and datasets.  

---

## Careers & Talent

While specific role listings weren’t provided, Hugging Face’s rapid expansion suggests a broad array of opportunities:

| Domain | Typical Roles |
|--------|---------------|
| **Machine Learning Engineering** | Model trainer, fine‑tune specialist, inference optimization. |
| **Full‑Stack Development** | Space dev, library maintainer, API/frontend engineer. |
| **Data Engineering & Operations** | Dataset curator, pipeline engineer, compute infra. |
| **Product & Growth** | Community manager, growth analyst, partner relations. |
| **Support & Consulting** | Enterprise support engineer, implementation specialist. |

*Apply via the platform’s **Sign Up**/careers section or LinkedIn.*  

---

## Join the Movement

- **Sign Up Free**: Create a personal portfolio, share projects, earn reputation.  
- **Contribute**: Submit models, datasets, or Space apps to the community.  
- **Explore Enterprise**: Request a demo or contact sales for a tailored solution.  

> **“Together, we’re building the most collaborative AI ecosystem on the planet.”**  

---

In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-oss:20b
Found 14 relevant links


# Hugging Face: The Place Where AI Gets a Big Friendly Hug  

> **“The AI community building the future.”**  
> *Because every great model deserves a warm welcome.*

---

## 🚀 What We Do  

| Category | What’s Happening | Why You’ll Love It |
|-----------|-----------------|--------------------|
| **Models** | 2 + million ready‑to‑go models (from 3D image tweaks to speech streaming). | Pick, tweak, or remix without the *oops* of your own code! |
| **Datasets** | 500 k+ datasets that cover text, image, video, audio, even 3D‑space. | Feed the next generation of ML – the universe’s data playground. |
| **Spaces** | Zero‑or‑zero‑extra‑GPU web apps you can spin up in a flash. | Deploy, demo, and share *without* paying for a GPU for a week. |
| **Community** | A bustling forum, Discord, and GitHub chats. | Real‑life coders, researchers, and coffee‑drinking philosophers all in one place. |

---

## Why Hugging Face?  

1. **Open‑source first** – We put the “repo” in “revolution.”  
2. **Zero‑GPU magic** – Run a GPU‑heavy demo in your browser (yes, *that* fast).  
3. **One‑stop shop** – Model → dataset → app ⇒ All in one hub.  
4. **Ethics + transparency** – We’re building an “open and ethical AI future together.”  

---

## The Culture (Short & Sweet)

- **“We’re the folks who hug each other’s code.”**  
  Pair‑programming? We call it *hug‑coding*.  
- **No “secret sauce,” just public sauce.**  
  Every repo is a playground; every pull request is a friendly handshake.  
- **Global, but local** – 30+ languages, 10+ time zones, one global community.  
- **“When in doubt, blame the model.”**  
  Because debugging is a group therapy session with your dataset.  

---

## Who’s Using Us?  

| Segment | How They Use Hugging Face | Tagline |
|---------|--------------------------|---------|
| **Enterprises** | Deploy secure, compliant inference endpoints with SSO and audit logs. | “AI for humans, not just robots.” |
| **Researchers** | Share pre‑trained checkpoints, benchmark, and replicate experiments. | “Reproducibility is our middle name.” |
| **Developers & Hobbyists** | Spin up an image generation app in a click. | “Why write nothing when you can generate everything?” |
| **Educators** | Build learning materials, assign datasets, and let students play. | “Teach, learn, repeat – while hugging all the models.” |

---

## Career Openings

> **“Join the squad that loves hugging more than scrolling.”**  
>  Explore the latest roles at the *Careers* page.  
>  Whether you’re a *NLP wizard*, *Data‑Ops ninja*, or *UI/UX champion*, we’ve got a spot for you.  

**Current Openings Include**:  
- Machine Learning Engineer  
- Platform Reliability Engineer  
- Community & Content Lead  

*(And there are more hidden around the site—like Easter eggs in a Christmas movie.)*

---

## Enterprise‑Level Features  

| Feature | Benefit | Why It Matters |
|---------|---------|----------------|
| **Single Sign‑On (SSO)** | One login for all your internal models. | Because the “Forgot Password” email is the worst. |
| **Region Selection & Audit Logs** | Pick where data lives, see who did what. | GDPR? HIPAA? We’re prepared. |
| **Resource Groups & Token Management** | Fine‑grained access, central approval. | “The only gatekeepers you’ll need.” |
| **Advanced Compute (ZeroGPU Quota Boost)** | 5× more GPU credits for every org member. | Make sure your models don’t have to sleep in the cloud. |
| **Private Dataset Viewer & 1 TB Private Storage** | Private datasets with full analytics. | Because secrets are good—but even better when you can explore them. |

---

## Quick‑Start: Hug It, Deploy it, Love it

1. **Sign up** – It takes less time than brewing your coffee.  
2. **Pick a model** – 2 M+ choices, you will never run out of options.  
3. **Create a Space** – Drag, drop, run.  
4. **Share** – One link and your friends are begging for your model.  

---

## Bottom Line  

Hugging Face is where **open‑source meets open‑heart**.  
We’re the friendly neighborhood library of AI where your code gets the hug it deserves.  

> **Join us** – and never leave an unsupervised model alone in the dark again.  
> **Build something amazing** – because the future is waiting for a big, warm hug.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>